In [1]:
import numpy as np

from sklearn.datasets import load_digits, load_iris
from sklearn.model_selection import train_test_split

In [2]:
rng = np.random.RandomState(1234567)
np.random.seed(1234567)

class KPerceptron(object):
    """
    X: nd*nx
    y: (nx,)
    """

    def __init__(self, kerntype='poly', kerngamma=1, kerncoef0=1, kerndegree=2,
            batch_size=100, ybias=0):
        self.kernfunc = getattr(self, '_kern_'+kerntype)
        self.kerngamma = kerngamma
        self.kerncoef0 = kerncoef0
        self.kerndegree = kerndegree
        self.batch_size = batch_size
        self.ybias = ybias

    def _kern_lin(self, xij):
        return np.dot(self.SV[self.svindex], xij)

    def _kern_poly(self, xij):
        dp = np.dot(self.SV[self.svindex], xij)
        return (self.kerngamma*dp+self.kerncoef0)**self.kerndegree

    def _kern_rbf(self, xij):
        """
        sv:  ns x nd
        xij: nd x nk
        """
        sv = self.SV[self.svindex]
        x_sq = (xij**2).sum(axis=0) # (nk,)
        sv_sq = (sv**2).sum(axis=1) # (ns,)
        return np.exp(-2*self.kerngamma * (x_sq + (-2*np.dot(sv,xij) + sv_sq.reshape(sv_sq.shape[0],1))))

    def __getYHat(self, xij, beta):
        if self.svindex.any():
            a = np.dot(beta[:,self.svindex], self.kernfunc(xij.T)) 
            return a.argmax(0)+1
        else:
            return np.ones(xij.shape[0],dtype=np.int)

    def __batch_fit(self, batch_indexes, Xtrn, ytrn):
        Xbatch, ybatch = Xtrn[batch_indexes,:], ytrn[batch_indexes]
        z = self.__getYHat(Xbatch,self.beta)
        updates = (z!=ybatch)
        self.beta2+=self.beta
        if updates.any():
            uInd = batch_indexes[updates]
            self.svindex[uInd] = True
            self.beta[ybatch[updates]-1,uInd]+=1
            self.beta[z[updates]-1,uInd]+=-1
            self.beta2+=self.beta

    def partial_fit(self, Xtrn, ytrn, classes=None):
        ytrn = ytrn + self.ybias
        if not hasattr(self, 'SV'):
            assert (np.unique(ytrn) == np.arange(1,ytrn.max()+1)).all()
            nx, nd, nc = Xtrn.shape[0], Xtrn.shape[1], ytrn.max()
            self.nx, self.nc = nx, nc
            self.beta = np.zeros((nc,nx),dtype=np.int) 
            self.beta2 = np.zeros((nc,nx),dtype=np.int) 
            self.svindex = np.zeros(nx, dtype=np.bool) 
            self.SV = Xtrn
            assert self.SV.shape == (nx,nd)
        indexes = np.random.permutation(self.nx)
        for i in np.arange(0, self.nx, self.batch_size): self.__batch_fit(indexes[i:min(i+self.batch_size,self.nx)], Xtrn, ytrn)

    def predict(self, X):
        return self.__getYHat(X,self.beta2) - self.ybias

    def score(self, X, y):
        y = y+self.ybias
        z = self.__getYHat(X,self.beta2)
        return np.sum(z==y)/float(y.size)

In [3]:
if __name__ == '__main__':
    dset = load_digits()
    print(dset.data.shape)
    Xtrn, Xtst, ytrn, ytst = train_test_split(dset.data, dset.target, test_size=0.3)
    Xtrn, Xtst, ytrn, ytst = Xtrn, Xtst,\
            ytrn+1, ytst+1
    print(Xtrn.shape, ytrn.shape, Xtst.shape, ytst.shape)
    print(np.unique(ytrn))

    sp = KPerceptron(kerntype='poly',kerngamma=1)
    for e in range(50):
        sp.partial_fit(Xtrn,ytrn)
        yHat, acc = sp.predict(Xtst), sp.score(Xtst,ytst)
        print('e: {} acc: {}'.format(e,acc))

(1797, 64)
(1257, 64) (1257,) (540, 64) (540,)
[ 1  2  3  4  5  6  7  8  9 10]
e: 0 acc: 0.6685185185185185
e: 1 acc: 0.8907407407407407
e: 2 acc: 0.9351851851851852
e: 3 acc: 0.9388888888888889
e: 4 acc: 0.9425925925925925
e: 5 acc: 0.9444444444444444
e: 6 acc: 0.95
e: 7 acc: 0.95
e: 8 acc: 0.95
e: 9 acc: 0.9518518518518518
e: 10 acc: 0.9537037037037037
e: 11 acc: 0.9537037037037037
e: 12 acc: 0.9537037037037037
e: 13 acc: 0.9555555555555556
e: 14 acc: 0.9555555555555556
e: 15 acc: 0.9574074074074074
e: 16 acc: 0.9574074074074074
e: 17 acc: 0.9574074074074074
e: 18 acc: 0.9592592592592593
e: 19 acc: 0.9592592592592593
e: 20 acc: 0.9611111111111111
e: 21 acc: 0.9629629629629629
e: 22 acc: 0.9629629629629629
e: 23 acc: 0.9629629629629629
e: 24 acc: 0.9629629629629629
e: 25 acc: 0.9629629629629629
e: 26 acc: 0.9629629629629629
e: 27 acc: 0.9629629629629629
e: 28 acc: 0.9629629629629629
e: 29 acc: 0.9629629629629629
e: 30 acc: 0.9629629629629629
e: 31 acc: 0.9648148148148148
e: 32 acc: 0.